In [1]:
import os
os.chdir('/Users/prana/Documents/MDS/Project-Group-13')

In [2]:
from projecttracker.utils import file_handler
import pandas as pd

In [14]:
class Operations:
    def __init__(self):
        self.projects = {}
        self.tasks = {}
        
    def view(self):
        # Read projects and tasks from JSON files
        projects_from_json = file_handler.read_from_json('project.json')
        tasks_from_json = file_handler.read_from_json('task.json')
        
        # Convert data to pandas DataFrame
        task_df = pd.DataFrame(tasks_from_json)
        project_df = pd.DataFrame(projects_from_json)

        # Merge the two DataFrames on 'projectID'
        result_df = pd.merge(project_df, task_df, on='projectID', how='left')

        # Display the result
        return result_df
        
    def add_proj(self, **kwargs):
        new_proj = Project(**kwargs)
        #self.projects[new_proj.projectID] = new_proj
        print(f"Project '{new_proj.projectID}' added.")
        file_handler.write_to_json(new_proj, 'project.json', 'a')  # Write to JSON file
        return new_proj

    def add_task(self, **kwargs):
        new_task = Task(**kwargs)
        #self.tasks[new_task.taskID] = new_task
        projects_from_json = file_handler.read_from_json('project.json')
        project_ids = [project['projectID'] for project in projects_from_json]
        
        if new_task.projectID in project_ids:
            print(f"Task '{new_task.taskID}' added.")
            file_handler.write_to_json(new_task, 'task.json', 'a')  # Write to JSON file
            return new_task
        else:
            print(f"Project {new_task.projectID} does not exist.")
            return None
    
    def modify_item(self):
        modify_type = input("Enter 'project' or 'task' ID to modify: ").upper()
        
        while True:
            # Read data from JSON files
            projects_from_json = file_handler.read_from_json('project.json')
            tasks_from_json = file_handler.read_from_json('task.json')

            # Identify the list and ID based on user input
            if modify_type[0] == 'P':
                items_list = projects_from_json
                item_ids = [project['projectID'] for project in projects_from_json]
            elif modify_type[0] == 'T':
                items_list = tasks_from_json
                item_ids = [task['taskID'] for task in tasks_from_json]
            else:
                print("Invalid input. Please enter correct 'project' or 'task' IDs.")
                break  # Exit the loop if the input is invalid

            # Check if the specified ID exists
            if modify_type not in item_ids:
                print(f"{modify_type} not found.")
                break  # Exit the loop if the ID is not found

            # Find the dictionary with the specified ID
            item_index = item_ids.index(modify_type)
            item = items_list[item_index]

            # Display current attributes
            print(f"Current attributes of {modify_type}:")
            for key, value in item.items():
                print(f"{key}: {value}")

            # Get the attribute to modify
            attribute = input("Enter the attribute to modify: ")

            # Check if the attribute exists
            if attribute not in item:
                print(f"Attribute '{attribute}' does not exist in {modify_type}.")
            else:
                # Get the new value for the attribute
                new_value = input(f"Enter new value for '{attribute}': ")

                # Update the attribute in the dictionary
                item[attribute] = new_value

                # Write the modified list of dictionaries back to the JSON file
                if modify_type[0] == 'P':
                    file_handler.delete_all_objects('project.json')
                    for project_item in projects_from_json:
                        file_handler.write_to_json_dict(project_item, 'project.json')
                elif modify_type[0] == 'T':
                    file_handler.delete_all_objects('task.json')
                    for task_item in tasks_from_json:
                        file_handler.write_to_json_dict(task_item, 'task.json')

                print(f"Attribute '{attribute}' updated for {modify_type} '{modify_type}'.")

            if input(f"Do you want to continue updating {modify_type} (Y/N)?").upper() != 'Y':
                break  # Exit the loop if the user does not want to continue
    
    def delete_item(self):
        projects_from_json = file_handler.read_from_json('project.json')
        project_ids = [project['projectID'] for project in projects_from_json]
        tasks_from_json = file_handler.read_from_json('task.json')
        task_ids = [task['taskID'] for task in tasks_from_json]
        
        delete_type = input("Enter 'project' or 'task' ID to delete: ").upper()
        
        if delete_type[0] == 'P':
            if delete_type in project_ids:
                project_list = [project for project in projects_from_json if project['projectID'] != delete_type]
                file_handler.delete_all_objects('project.json')
                for project_item in project_list:
                    file_handler.write_to_json_dict(project_item, 'project.json')
                print(f"Project '{delete_type}' deleted.")
                return delete_type
            else:
                print(f"Project '{delete_type}' not found.")
                return None
        elif delete_type[0] == 'T':
            if delete_type in task_ids:
                task_list = [task for task in tasks_from_json if task['taskID'] != delete_type]
                file_handler.delete_all_objects('task.json')
                for task_item in task_list:
                    file_handler.write_to_json_dict(task_item, 'task.json')
                print(f"Task '{delete_type}' deleted.")
                return delete_type
            else:
                print(f"Task '{delete_type}' not found.")
                return None
        else:
            print("Invalid input. Please enter 'project' or 'task'.")
            return None
    

class Project(Operations):
    #project_counter = 0
    def __init__(self, Name, Priority, Duration, Comments, assignedTo, startDate, Deadline, Owner):
        project_id = self.get_next_project_id()
        self.projectID = f'P{project_id:04}'
        self.projectName = Name
        self.projectPriority = Priority
        self.projectDuration = Duration
        self.projectComments = Comments
        self.assignedToProjectTL = assignedTo
        self.projectStartDate = startDate 
        self.projectDeadline = Deadline
        self.projectOwner = Owner
        self.projectStatus = 'Not Started'
    
    def get_next_project_id(self):
        last_project_id = 0
        try:
            projects_from_json = file_handler.read_from_json('project.json')
        except:
            pass
        else:
            project_ids = [project['projectID'] for project in projects_from_json]
            last_inserted_project_id = max(project_ids, key=lambda x: int(x[1:]))
            last_project_id = int(last_inserted_project_id[1:])
        finally:
            last_project_id += 1
            return last_project_id
    


class Task(Project):
    #task_counter = 0
    def __init__(self, projectID, Name, Priority, Duration, Comments, assignedTo, startDate, Deadline):
        self.projectID = projectID
        task_id = self.get_next_task_id()
        self.taskID = f'T{task_id:04}'
        self.taskName = Name
        self.taskPriority = Priority
        self.taskDuration = Duration
        self.taskComments = Comments
        self.assignedToTask = assignedTo
        self.taskStartDate = startDate
        self.taskDeadline = Deadline
        self.taskStatus = 'Not Started'
    
    def get_next_task_id(self):
        last_task_id = 0
        try:
            tasks_from_json = file_handler.read_from_json('task.json')
        except:
            pass
        else:
            task_ids = [task['taskID'] for task in tasks_from_json]
            last_inserted_task_id = max(task_ids, key=lambda x: int(x[1:]))
            last_task_id = int(last_inserted_task_id[1:])
        finally:
            last_task_id += 1
            return last_task_id
    
    



In [16]:
operations = Operations()
operations.modify_item()

Enter 'project' or 'task' ID to modify: P0005
Current attributes of P0005:
projectID: P0005
projectName: Project 1
projectPriority: High
projectDuration: 2 weeks
projectComments: Important project
assignedToProjectTL: Team Lead 1
projectStartDate: 2023-11-11
projectDeadline: 2023-11-30
projectOwner: Project Owner 1
projectStatus: Not Started
Enter the attribute to modify: projectName
Enter new value for 'projectName': Proj 5
Attribute 'projectName' updated for P0005 'P0005'.
Do you want to continue updating P0005 (Y/N)?N


In [59]:
operations = Operations()

project_data = {
    'Name': 'Project 1',
    'Priority': 'High',
    'Duration': '2 weeks',
    'Comments': 'Important project',
    'assignedTo': 'Team Lead 1',
    'startDate': '2023-11-11',
    'Deadline': '2023-11-30',
    'Owner': 'Project Owner 1',
}
operations.add_proj(**project_data)

# Add a task
task_data = {
    'projectID': 'P0002',
    'Name': 'Task 1',
    'Priority': 'Medium',
    'Duration': '1 week',
    'Comments': 'Urgent task',
    'assignedTo': 'Team Member 1',
    'startDate': '2023-11-02',
    'Deadline': '2023-12-28',
}
operations.add_task(**task_data)

# Modify a project or task
#operations.modify_item()

# Delete a project or task
#operations.delete_item()

# Read projects and tasks from JSON files
projects_from_json = file_handler.read_from_json('project.json')
tasks_from_json = file_handler.read_from_json('task.json')
#tasks_from_json = read_from_json()

# Display the read projects and tasks
print("\nProjects read from JSON:")
for project_data in projects_from_json:
    print(project_data)

print("\nTasks read from JSON:")
for task_data in tasks_from_json:
    print(task_data)
    
#operations.delete_item()

Project 'P0006' added.
Task 'T0005' added.

Projects read from JSON:
{'projectID': 'P0001', 'projectName': 'Project 1', 'projectPriority': 'high', 'projectDuration': '5', 'projectComments': 'test', 'assignedToProjectTL': 'Pranav', 'projectStartDate': '2023-11-12', 'projectDeadline': '2023-12-23', 'projectOwner': 'Natalie', 'IsProjectCompleted': 'N'}
{'projectID': 'P0002', 'projectName': 'Project 1', 'projectPriority': 'High', 'projectDuration': '2 weeks', 'projectComments': 'Important project', 'assignedToProjectTL': 'Team Lead 1', 'projectStartDate': '2023-11-11', 'projectDeadline': '2023-11-30', 'projectOwner': 'Project Owner 1', 'projectStatus': 'Not Started'}
{'projectID': 'P0003', 'projectName': 'Project 1', 'projectPriority': 'High', 'projectDuration': '2 weeks', 'projectComments': 'Important project', 'assignedToProjectTL': 'Team Lead 1', 'projectStartDate': '2023-11-11', 'projectDeadline': '2023-11-30', 'projectOwner': 'Project Owner 1', 'projectStatus': 'Not Started'}
{'projec

In [17]:
df = operations.view()
df.head(10)

,projectID,projectName,projectPriority,projectDuration,projectComments,assignedToProjectTL,projectStartDate,projectDeadline,projectOwner,projectStatus,taskID,taskName,taskPriority,taskDuration,taskComments,assignedToTask,taskStartDate,taskDeadline,taskStatus
0,P0002,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P0004,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P0005,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P0006,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P0007,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,P0008,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,P0009,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,P0010,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,P0011,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,P0012,Project 1,High,2 weeks,Important project,Team Lead 1,2023-11-11,2023-11-30,Project Owner 1,Not Started,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
projects_from_json

In [ ]:
project_ids = [project['projectID'] for project in projects_from_json]

    # Find the last inserted projectID
last_inserted_project_id = max(project_ids, key=lambda x: int(x[1:]))

In [ ]:
int(last_inserted_project_id[1:])

In [ ]:
import pandas as pd
# Convert data to pandas DataFrame
task_df = pd.DataFrame(tasks_from_json)
project_df = pd.DataFrame(projects_from_json)

# Merge the two DataFrames on 'projectID'
result_df = pd.merge(project_df, task_df, on='projectID', how='left')

# Display the result
result_df.head()

In [ ]:
project_id_input = input("Enter the Project ID: ")


In [ ]:
data = []
tasks_from_json = file_handler.read_from_json('task.json')
task_list = [task for task in tasks_from_json if task['projectID'] == project_id_input]

for x in task_list:
    data.append(dict(Task=x['taskName'], Start=x['taskStartDate'], Finish=x['taskDeadline'], Priority=x['taskPriority']))

data

In [ ]:

# Create a Gantt chart
fig = px.timeline(data, x_start="Start", x_end="Finish", y="Task", hover_data={"Priority": True}, labels={"Task": f"Project: {project_id_input}"})

# Current date
current_date = datetime.datetime.now().strftime('%Y-%m-%d')

# Add a vertical line for the current date
fig.update_layout(shapes=[
    dict(
        type="line",
        x0=current_date,
        x1=current_date,
        y0=0,
        y1=1,
        yref="paper",
        line=dict(color="black", width=2),
    )
])

# Show the Gantt chart
fig.show()

In [ ]:
data = []
projects_from_json = file_handler.read_from_json('project.json')
project_list = [project for project in projects_from_json]
for x in project_list:
        data.append(dict(Project=x["projectName"], Start=x["projectStartDate"], Finish=x["projectDeadline"], Priority=x["projectPriority"]))


# Create a Gantt chart
fig = px.timeline(data, x_start="Start", x_end="Finish", y="Project", hover_data={"Priority": True})



# Show the Gantt chart
fig.show()

In [ ]:
data = []
tasks_from_json = file_handler.read_from_json('task.json')
task_list = [task for task in tasks_from_json if task['projectID'] == project_id_input]

for x in task_list:
    data.append(dict(Task=x['taskName'], Status=x['taskStatus']))

data

In [ ]:
import plotly.express as px
# Create Pie Chart for Task Status with a lighter red for 'Completed' status
fig = px.pie(data, names='Status', title='Task Status Distribution',
             color_discrete_map={'Completed': 'lightcoral', 'In Progress': 'blue', 'Not Started': 'green'})

# Show the Pie Chart
fig.show()

In [23]:
def write_list_of_dicts_to_csv(data, file_path):
    data.to_csv(file_path, index=False)

In [25]:
write_list_of_dicts_to_csv(df, '/Users/prana/Documents/df.csv')

In [61]:

    # Create a DataFrame from the list of tasks
df = pd.DataFrame(task_list)

    # Group by project and count the number of tasks in each project
project_counts = df.groupby('projectID').size().reset_index(name='Number of Tasks')

    # Create a bar plot
fig = px.bar(project_counts, x='projectID', y='Number of Tasks', title='Number of Tasks in Each Project')

    # Update layout
fig.update_layout(xaxis_title='Project', yaxis_title='Number of Tasks', showlegend=False)

    # Show the figure
fig.show()

In [60]:
#data = []
tasks_from_json = file_handler.read_from_json('task.json')
task_list = [task for task in tasks_from_json]

task_list

[{'projectID': 'P0001',
  'taskID': 'T0001',
  'taskName': 'Task 1',
  'taskPriority': 'Medium',
  'taskDuration': '1 week',
  'taskComments': 'Urgent task',
  'assignedToTask': 'Team Member 1',
  'taskStartDate': '2023-11-02',
  'taskDeadline': '2023-12-28',
  'taskStatus': 'Not Started'},
 {'projectID': 'P0001',
  'taskID': 'T0002',
  'taskName': 'Task 1',
  'taskPriority': 'Medium',
  'taskDuration': '1 week',
  'taskComments': 'Urgent task',
  'assignedToTask': 'Team Member 1',
  'taskStartDate': '2023-11-02',
  'taskDeadline': '2023-12-28',
  'taskStatus': 'Not Started'},
 {'projectID': 'P0001',
  'taskID': 'T0003',
  'taskName': 'Task 1',
  'taskPriority': 'Medium',
  'taskDuration': '1 week',
  'taskComments': 'Urgent task',
  'assignedToTask': 'Team Member 1',
  'taskStartDate': '2023-11-02',
  'taskDeadline': '2023-12-28',
  'taskStatus': 'Not Started'},
 {'projectID': 'P0001',
  'taskID': 'T0004',
  'taskName': 'Task 1',
  'taskPriority': 'Medium',
  'taskDuration': '1 week',